In [2]:
import jsonlines
import re
import datasets
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

def unpack_data():    
    with jsonlines.open('test.jsonl') as reader1, jsonlines.open('train.jsonl') as reader2, jsonlines.open('val.jsonl') as reader3:
        test_data = construcked_data(reader1)
        train_data = construcked_data(reader2)
        val_data = construcked_data(reader3)
        return test_data, train_data, val_data
              
def construcked_data(data):
        train_data={}
        questions = []
        answer_labeled = []
        tmpDict=[]
        for deep in data:     
            for deeper in deep.get('passage').get('questions'):    
                for deepest in deeper['answers']:
                    if "label" in deepest and deepest.get('label') == 1:
                        answer_labeled.append({"answer": deepest.get('text'), "label": deepest.get('label')})
                    elif "label" not in deepest:
                        answer_labeled.append({"answer": deepest.get('text')})
                questions.append({"question": deeper.get('question'), "answers": answer_labeled})
                answer_labeled= []
            passage = deep.get('passage').get('text')
            passage = re.sub(r'\([-+]?\d+\)', '', passage)
            tmpDict.append({"idx": deep.get('idx'), "passage": passage , "questions": questions})      
            questions=[] 
        
        return(tmpDict)
        
test_data, train_data, val_data = unpack_data()



    
def find_start_pos(data):
    
    morph = MorphAnalyzer()
    russian_stopwords = stopwords.words("russian")
    
    def token_analizer(idx,token_passage,token_answer):     
        indexes_ans = []
        startpoz = 0
        endpoz = 0
        
        indexes_ans.append([i for i, x in enumerate(token_passage) if x == token_answer[0]])
        indexes_ans.append([i for i, x in enumerate(token_passage) if x == token_answer[-1]])
        if len(indexes_ans[0])>0 and len(indexes_ans[-1])>0:
            for j in indexes_ans[0]:
                for d in indexes_ans[-1]:
                    if len(token_answer)-2 < d-j and len(token_answer)+3 > d-j:
                        startpoz = j
                        endpoz = d
                    break
                
                    
                    
                  
            
        
        #print(token_passage,token_answer)
        #print(indexes_ans) 
        #print(startpoz)
        #print(idx)
        return startpoz, endpoz
        
    def make_token(text):
        tokened = []
        
        for token in text.split():
            token = token.strip().strip('.')
            token = morph.normal_forms(token)[0]
            tokened.append(token)
        
        return tokened
    
    def untokened_label(text, label_s, label_e):
        if label_s != 0:
            tokened = []
            tx = 0
            tx2 = 0
            for token in text.split():
                tokened.append(token)
            for i in range(label_s):
                tx+=(len(tokened[i])+1)
            for i in range(label_e):
                tx2+=(len(tokened[i])+1)
            index = text.find(tokened[label_s], tx)
            index1 = text.find(tokened[label_s], tx, tx2)
            if index == index1 or index1 == -1:
                return index
            
            
        
        else:
            return None
            
    
    
    
    for deep in data:
        
        passage = deep.get('passage')
        
        passage = re.sub(r'[^\w\s]','', passage) 
        token_passage = make_token(passage)
        for deeper in deep.get('questions'):
            for deepest in deeper['answers']:
                answer = re.sub(r'[^\w\s]','', deepest.get('answer')) 
                token_answer=make_token(answer)
                st, ed = token_analizer(deep.get('idx'),token_passage,token_answer)
                label = untokened_label(deep.get('passage'),st, ed)
                deepest['label'] = label
                
    
        token_passage=[]
    
    
        
                    
    
find_start_pos(train_data)

find_start_pos(val_data)
    
    


In [3]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        if answer["answer_start"][0] != None:
            
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)
    
            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1
    
            # If the answer is not fully inside the context, label is (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)
    
                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)
        else:
            start_positions.append(0)
            end_positions.append(0)
                  

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []
    
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs



In [4]:
import torch
from transformers import AutoTokenizer
from datasets import Dataset
trained_checkpoint = "timpal0l/mdeberta-v3-base-squad2/checkpoint-3603"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
ans = str
label = []
a=[]
idx = 0
for deep in val_data:     
        for deeper in deep.get('questions'):        
            for deepest in deeper['answers']:
                ans=deepest.get('answer')
                label= deepest.get('label')
                
                a.append({'id':str(idx),'context':deep.get('passage'),'question':deeper.get('question'), 'answers':ans, 'answer_start':label})
                idx+=1
                ans = str
                label = str

valid_dataset_premade = Dataset.from_list(a)



ans = []
label = []
a=[]
idx = 0
for deep in train_data:     
        for deeper in deep.get('questions'):        
            for deepest in deeper['answers']:
                ans.append(deepest.get('answer'))
                label.append(deepest.get('label'))
                a.append({'id':str(idx),'context':deep.get('passage'),'question':deeper.get('question'), 'answers':{"text":ans , 'answer_start':label}})
                ans = []
                label = []
                idx+=1

train_dataset_pre = Dataset.from_list(a)


train_dataset = train_dataset_pre.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_dataset_pre.column_names,
)






eval_set = valid_dataset_premade.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns = valid_dataset_premade.column_names,
)


Map:   0%|          | 0/5380 [00:00<?, ? examples/s]

Map:   0%|          | 0/993 [00:00<?, ? examples/s]

In [5]:
from tqdm.auto import tqdm
import numpy as np
import evaluate
import collections
metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30
theoretical_answers = []
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})
    for ex in examples:
        theoretical_answers.append({"id": ex["id"], "answers":[{"text": ex["answers"], "answer_start": ex['answer_start']}]}) 
    print(len(theoretical_answers[0]['answers']))
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
from transformers import TrainingArguments
import torch
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
from transformers import Trainer

model_checkpoint = "timpal0l/mdeberta-v3-base-squad2/checkpoint-3603"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

args = TrainingArguments(
    "timpal0l/mdeberta-v3-base-squad2/checkpoint-3603",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_set,
    tokenizer=tokenizer,
)
predictions, _, _ = trainer.predict(eval_set)
start_logits, end_logits = predictions

compute_metrics(start_logits, end_logits, eval_set, valid_dataset_premade)


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
